<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebooks/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

In [7]:
import numpy as np
# import torch
import matplotlib.pyplot as plt

In [8]:
# dim=2000000
# a = 100*np.random.randn(4, dim) + 1123
# g = np.random.randn(dim, 1)
# b = (a-np.mean(a, axis=1)[:, np.newaxis])/np.std(a, axis=1)[:, np.newaxis]
# np.mean(b@g/dim)

In [37]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01, eff=0.1):
    def log():
        print(f"Iter={iter}, Condition={np.min(dgr@g/dim)}")

    dgr = domain_grads
    number_of_domains, dim = dgr.shape

    #
    _mean = np.mean(dgr, axis=1)[:, np.newaxis]
    _std = np.std(dgr, axis=1)[:, np.newaxis]
    dgr = (dgr-_mean)/_std
    
    #
    g = np.random.randn(dim)
    u_ = np.zeros(number_of_domains)
    
    iter = 0
    log()
    while not np.min(dgr@g/dim) >= epsilon:
        if iter >= 100:
            break

        u_ = u_ + alpha*((1.+eff)*epsilon - (dgr@g)/dim)
        g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)

        iter += 1
        log()
    return g

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [32]:
epsilon=10
alpha=1

In [54]:
grads = 1*np.random.randn(20, 20000) - 10

In [55]:
g = gradient_maker(grads, epsilon=epsilon, alpha=alpha)

Iter=0, Condition=-0.01258394341344263
Iter=1, Condition=0.5720288081472397
Iter=2, Condition=1.0689631236491113
Iter=3, Condition=1.5422400449210802
Iter=4, Condition=1.992984516608491
Iter=5, Condition=2.4222680629284055
Iter=6, Condition=2.8311113204459892
Iter=7, Condition=3.2204864509883344
Iter=8, Condition=3.5913194403559188
Iter=9, Condition=3.944492288225265
Iter=10, Condition=4.280845094382256
Iter=11, Condition=4.601178046183491
Iter=12, Condition=4.906253311912163
Iter=13, Condition=5.196796844475145
Iter=14, Condition=5.473500099678229
Iter=15, Condition=5.737021673116889
Iter=16, Condition=5.987988859529365
Iter=17, Condition=6.2269991382776935
Iter=18, Condition=6.454621588449238
Iter=19, Condition=6.671398236906619
Iter=20, Condition=6.877845342457005
Iter=21, Condition=7.0744546191618936
Iter=22, Condition=7.261694401666316
Iter=23, Condition=7.440010755290094
Iter=24, Condition=7.609828533494713
Iter=25, Condition=7.771552385215712
Iter=26, Condition=7.925567714433098

In [56]:
mean_ = np.mean(grads, axis=1)[:, np.newaxis]
std_ = np.std(grads, axis=1)[:, np.newaxis]
eps = epsilon
d = grads.shape[1]
g_bar = np.mean(g)

In [57]:
(grads@g)[:, np.newaxis]>=eps*d*std_+d*g_bar*mean_

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])

In [58]:
eps*d*std_+d*g_bar*mean_>=eps*d

array([[False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True]])